In [ ]:
import logging
from qiskit_ibm_runtime import QiskitRuntimeService

from sqlalchemy import select, func

from benchmarklib import BaseProblem, CompileType, BaseTrial, BenchmarkDatabase
from benchmarklib.compilers import XAGCompiler, QCFCompiler
from benchmarklib.core.database import logger as db_logger

from rbf import run_rbf_benchmark, generate_rbf_problems, RandomBooleanFunction, RandomBooleanFunctionTrial
from rbf import logger as rbf_logger
rbf_logger.setLevel(logging.INFO)
db_logger.setLevel(logging.INFO)
handler = logging.StreamHandler()
handler.setLevel(logging.INFO)
rbf_logger.addHandler(handler)
rbf_logger.propagate = False
db_logger.addHandler(handler)
db_logger.propagate = False


In [ ]:
service = QiskitRuntimeService()
backend = service.backend(name="ibm_rensselaer")
benchmark_db = BenchmarkDatabase("rbf.db", RandomBooleanFunction, RandomBooleanFunctionTrial)

def create_compilation_failure(problem_id, compiler_name):
    problem = benchmark_db.get_problem_instance(problem_id)
    trial = RandomBooleanFunctionTrial(
        problem=problem,
        compiler_name=compiler_name,
        is_failed=True,
        input_state=""
    )
    benchmark_db.save_trial(trial)

def mark_job_failure(db, job_id):
    """
    Call this if a job failed due to internal error with IBM
    """
    trials = db.find_trials(job_id=job_id, include_pending=True)
    with db.session() as session:
        for trial in trials:
            if trial.is_pending:
                trial.is_failed = True
                session.merge(trial)
        session.commit()

In [ ]:
# run this cell once (will have no effect if already run) to populate the database with problem instances
generate_rbf_problems(
    benchmark_db,
    num_vars_iter=range(2, 21),
    complexity_iter=range(1, 21),
    num_functions=20
)

In [ ]:


run_rbf_benchmark(
    benchmark_db,
    QCFCompiler(),
    backend=backend,
    num_vars_iter=range(20, 21),
    complexity_iter=range(1, 21),
    num_functions=20,
    trials_per_instance=5,
    shots=10**3,
    max_problems_per_job=4,
    save_circuits=False
)

In [ ]:
failed_problem_id  = 8319
create_compilation_failure(
    failed_problem_id,
    QCFCompiler().name
)
print(failed_problem_id)

In [ ]:
mark_job_failure(benchmark_db, "d3s56bmjbaoc73cn01d0")

In [ ]:
await benchmark_db.update_all_pending_results(service, batch_size=1)

In [ ]:
# delete trials that did not submit
for trial in benchmark_db.query(
    select(RandomBooleanFunctionTrial)
        .where(
            RandomBooleanFunctionTrial.compiler_name == QCFCompiler().name, 
            RandomBooleanFunctionTrial.job_id == None, 
            RandomBooleanFunctionTrial.is_failed == False
        )
    ):
    print(trial.id)
    benchmark_db.delete_trial(trial.trial_id)

In [ ]:
count = benchmark_db.query(select(func.count(RandomBooleanFunctionTrial.id)).where(RandomBooleanFunctionTrial.compiler_name == compiler.name))[0]
print(f"Number of {compiler.name} trials in db: {count}")